In [2]:
import pandas as pd
import sys
import os
import datetime
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import InputLayer, Dense
import keras.optimizers as optimisers
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
sys.path.insert(0, os.path.abspath('../')) 

from lib.common import constants as k
from lib.eirgrid import data as eirgrid_data
from lib.common.marginal_emissions import compute_mef
from lib.common.data_window import DataWindow
import lib.common.gra as gra

In [3]:
eirgrid = eirgrid_data.system()
eirgrid = compute_mef(eirgrid).dropna()
eirgrid

,EffectiveTime,SysFrequency,Co2Emissions,Co2Intensity,SystemDemand,GenExp,InterNet,WindActual,MarginalEmissions
56,2023-12-28 09:30:00,49.98,676.0,146.0,4638.0,5298.0,-237.0,3798.0,-201.200000
57,2023-12-28 09:15:00,49.99,692.0,152.0,4555.0,5208.0,-213.0,3697.0,-1334.000000
58,2023-12-28 09:00:00,50.02,687.0,154.0,4473.0,5201.0,-288.0,3719.0,729.888889
59,2023-12-28 08:45:00,50.01,667.0,152.0,4385.0,5183.0,-770.0,3723.0,-67.756250
60,2023-12-28 08:30:00,49.95,682.0,159.0,4295.0,5023.0,-290.0,3511.0,159.000000
...,...,...,...,...,...,...,...,...,...
122476,2020-07-01 01:00:00,50.06,1506.0,414.0,3249.0,3542.0,-120.0,246.0,2834.419355
122477,2020-07-01 00:45:00,50.09,1600.0,435.0,3297.0,3573.0,-102.0,231.0,625.063694
122478,2020-07-01 00:30:00,50.07,1698.0,443.0,3376.0,3730.0,0.0,239.0,358.321168
122479,2020-07-01 00:15:00,50.02,1747.0,440.0,3453.0,3867.0,-235.0,243.0,12.333333


In [4]:
fuel_mix = pd.read_csv(k.PROCESSED_DATA_DIR / 'fuel_mix.csv', index_col=0)
fuel_mix.rename(columns={'StartTime': 'EffectiveTime'}, inplace=True)
fuel_mix['EffectiveTime'] = pd.to_datetime(fuel_mix['EffectiveTime'])
             
pricing  = pd.read_csv(k.RAW_DATA_DIR / 'semo' / 'price_all_latest.csv', index_col=0)
pricing = pricing[['StartTime', 'NetImbalanceVolume', 'ImbalanceSettlementPrice']]
pricing.rename(columns={'StartTime': 'EffectiveTime'}, inplace=True)
pricing['EffectiveTime'] = pd.to_datetime(pricing['EffectiveTime'])

combined_sys = eirgrid.merge(fuel_mix, on='EffectiveTime', how='left').dropna()
# combined_sys = combined_sys.merge(pricing, on='EffectiveTime', how='left').dropna()
combined_sys.shape, combined_sys.dtypes

((57716, 22),
 EffectiveTime                datetime64[ns]
 SysFrequency                        float64
 Co2Emissions                        float64
 Co2Intensity                        float64
 SystemDemand                        float64
 GenExp                              float64
 InterNet                            float64
 WindActual                          float64
 MarginalEmissions                   float64
 Fuel_Battery_Storage                float64
 Fuel_Biomass_Peat                   float64
 Fuel_Coal                           float64
 Fuel_Distillate                     float64
 Fuel_Gas                            float64
 Fuel_Oil                            float64
 Fuel_Other_Non_Renewable            float64
 Fuel_Other_Renewable                float64
 Fuel_Pumped_Storage_Hydro           float64
 Fuel_Renewable_Hydro                float64
 Fuel_Solar                          float64
 Fuel_Waste                          float64
 Fuel_Wind                           floa

Setup training testing data based on entire data set

In [15]:
df = gra.dt_encode(combined_sys, column='EffectiveTime')

scaled_df = df.copy()
scaler = MinMaxScaler()
scaled_df[scaled_df.columns] = scaler.fit_transform(df)

target_name = 'Co2Intensity'
features = scaled_df.drop(columns=['Co2Intensity', 'MarginalEmissions'])
target   = scaled_df[target_name]
grg = gra.grg(features, target)
feature_set = gra.relevant_features(grg)
grg.mean()

Fuel_Gas                     0.798821
Co2Emissions                 0.789493
Fuel_Coal                    0.775485
Fuel_Other_Non_Renewable     0.774422
InterNet                     0.730661
SystemDemand                 0.728555
GenExp                       0.724184
SysFrequency                 0.716700
Fuel_Biomass_Peat            0.712249
Fuel_Other_Renewable         0.693633
Fuel_Renewable_Hydro         0.663527
Fuel_Oil                     0.643374
EffectiveTime_cos            0.638806
EffectiveTime_sin            0.635105
WindActual                   0.627842
Fuel_Wind                    0.622192
Fuel_Pumped_Storage_Hydro    0.612199
Fuel_Waste                   0.583358
Fuel_Distillate              0.582288
Fuel_Solar                   0.578703
Fuel_Battery_Storage         0.572848
dtype: float64

In [16]:
pca = PCA(n_components=0.95)
pca_components = pca.fit_transform(features)
pca_variances = pca.explained_variance_ratio_
combined_variance = np.sum(pca.explained_variance_ratio_)*100
print('Using %d/%d components %.2f%% of the variance is retained' % (len(pca_variances), len(features.columns), combined_variance))

n_pcs = pca.n_components_
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
initial_feature_names = features.columns
most_important_features = [(i, initial_feature_names[most_important[i]], pca.components_[i][most_important[i]]) for i in range(n_pcs)]

print('The most important features from the PCA analysis are')
most_important_cols = list(set(map(lambda x: x[1], most_important_features)))
most_important_features, most_important_cols

Using 13/21 components 96.28% of the variance is retained
The most important features from the PCA analysis are


([(0, 'EffectiveTime_cos', -0.46910641422913146),
  (1, 'Fuel_Wind', 0.5020495239680475),
  (2, 'EffectiveTime_sin', -0.7105544235435685),
  (3, 'Fuel_Coal', 0.5895372694992044),
  (4, 'Fuel_Renewable_Hydro', 0.83364241092466),
  (5, 'Fuel_Waste', -0.9533100373452801),
  (6, 'InterNet', -0.8650074823175244),
  (7, 'Fuel_Pumped_Storage_Hydro', 0.8090980539753483),
  (8, 'Fuel_Pumped_Storage_Hydro', -0.4751572443775151),
  (9, 'Fuel_Biomass_Peat', -0.5561519764078116),
  (10, 'Fuel_Oil', 0.6450711398426984),
  (11, 'Fuel_Solar', 0.5616172668736482),
  (12, 'Fuel_Solar', 0.7266948290977884)],
 ['InterNet',
  'Fuel_Waste',
  'Fuel_Biomass_Peat',
  'Fuel_Coal',
  'Fuel_Wind',
  'Fuel_Renewable_Hydro',
  'EffectiveTime_cos',
  'EffectiveTime_sin',
  'Fuel_Oil',
  'Fuel_Solar',
  'Fuel_Pumped_Storage_Hydro'])

Setup train/test split

In [18]:
# Create feature columns
X = df[most_important_cols]
Y = df['Co2Intensity']

# Train test split with 20% test size
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize the input data
scaler  = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

Use Keras for MLP network

In [21]:
model = Sequential()

# Setup the input layer with 8 input features
model.add(InputLayer(input_shape=(X_train.shape[1], ), name="features"))

# Setup two hidden layers
# The first hidden layer will have 64 neurons (arbitrarily chosen)
model.add(Dense(128, activation="relu", name="hidden_1"))

# The second hidden layer will have 64 neurons
model.add(Dense(64, activation="relu", name="hidden_2")) 

# This will predict MEF and CO2 Intensity so two output layers
# Use a linear activation function for the output so we don't restirct or tamper the value
model.add(Dense(1, activation="linear", name="target"))

# Use adam optimiser to start
# Measure losses using MSE
model.compile(loss='mse', optimizer=optimisers.legacy.Adadelta(learning_rate=1))

# Summarise the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_1 (Dense)            (None, 512)               6144      
                                                                 
 hidden_2 (Dense)            (None, 256)               131328    
                                                                 
 target (Dense)              (None, 1)                 257       
                                                                 
Total params: 137729 (538.00 KB)
Trainable params: 137729 (538.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(X_train, Y_train, epochs=10000, validation_split=0.2)

Epoch 1/10000
1155/1155 [==============================] - 1s 1ms/step - loss: 2821.1172 - val_loss: 1323.9957
Epoch 2/10000
1155/1155 [==============================] - 1s 958us/step - loss: 1209.0123 - val_loss: 1300.3687
Epoch 3/10000
1155/1155 [==============================] - 1s 1ms/step - loss: 1170.5896 - val_loss: 1350.1226
Epoch 4/10000
1155/1155 [==============================] - 1s 936us/step - loss: 1148.3231 - val_loss: 1391.9777
Epoch 5/10000
1155/1155 [==============================] - 1s 993us/step - loss: 1130.2542 - val_loss: 1360.0221
Epoch 6/10000
1155/1155 [==============================] - 2s 1ms/step - loss: 1115.2350 - val_loss: 1097.2795
Epoch 7/10000
1155/1155 [==============================] - 2s 2ms/step - loss: 1096.3685 - val_loss: 1969.6208
Epoch 8/10000
1155/1155 [==============================] - 1s 1ms/step - loss: 1082.4026 - val_loss: 1083.9574
Epoch 9/10000
1155/1155 [==============================] - 1s 1ms/step - loss: 1068.7537 - val_loss: 1090.

KeyboardInterrupt: 

In [128]:
model.evaluate(X_test, Y_test)

750/750 [==============================] - 0s 280us/step - loss: 2699.4360


2699.43603515625